In [2]:
!pip install evaluate
!pip install rouge_score

import pandas as pd
import torch
import os
import datasets
import evaluate

from tqdm import tqdm
from transformers import BartForConditionalGeneration, BartTokenizer , pipeline
from torch.utils.data import DataLoader


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
#### Base BART
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [4]:
df = pd.read_csv(os.path.join(os.getcwd(),'data/new_summarization_data.csv'))
print(df.shape)

(870521, 5)


In [5]:
df = df.head(300)

In [6]:
df.head(1)

,Unnamed: 0,ID,Content,Summary,Dataset
0,0,f49ee725a0360aa6881ed1f7999cc531885dd06a,New York police are concerned drones could bec...,Police have investigated criminals who have ri...,CNN/Daily Mail


### BART FT

In [7]:
from datasets import Dataset, DatasetDict

df_1 = df.drop(columns={'Unnamed: 0' , 'ID' , 'Dataset'}, axis=1)

def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True).input_ids

df_train = df_1.sample(frac=0.8,random_state=12)
df_test = df_1.drop(df_train.index)

train = pd.DataFrame(index=None)

train['input_ids'] = df_train['Content'].apply(tokenize_function)
train['labels'] = df_train['Summary'].apply(tokenize_function)

test = pd.DataFrame(index=None)

test['input_ids'] = df_test['Content'].apply(tokenize_function)
test['labels'] = df_test['Summary'].apply(tokenize_function)

datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
    })

datasets_train_test = datasets_train_test.remove_columns(['__index_level_0__'])

datasets_train_test.set_format('torch')

train_dataloader = DataLoader(datasets_train_test['train'], shuffle=True, batch_size=4)
eval_dataloader = DataLoader(datasets_train_test['test'], batch_size=4)

In [8]:
train_dataloader.dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 240
})

In [9]:
from torch.optim import AdamW
# fine-tuning BART
optimizer = AdamW(model.parameters() , lr=5e-5)
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
device = torch.device("cpu")
print(device)
model.to(device)

cpu


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [10]:
from tqdm.auto import tqdm

model.train()

for epoch in range(num_epochs):
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        optimizer.zero_grad()
        loss = outputs.loss
        loss.backward()
        optimizer.step()

  0%|          | 0/60 [00:00<?, ?it/s]

In [11]:
### Simple ft BART
model.save_pretrained(os.getcwd()+'/models/ft_BART_news_summarize')

### FT and PT comparison

In [12]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

model_ft = BartForConditionalGeneration.from_pretrained(os.getcwd() + '/models/ft_BART_news_summarize')
summarization_pipeline_ft = pipeline("summarization", model_ft, tokenizer=tokenizer)

model_base = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
summarization_pipeline_base = pipeline("summarization", model_base, tokenizer=tokenizer)

In [13]:
# {'rouge1': 0.16258091442735179, 'rouge2': 0.040756608170613315, 'rougeL': 0.12557953810045613, 'rougeLsum': 0.1404810811938183} <- max_length = 20
# {'rouge1': 0.22725943712723934, 'rouge2': 0.06288716574190041, 'rougeL': 0.14955314802409142, 'rougeLsum': 0.18272286049547903 <- max_length = 100
predictions = []
ground_truth = df['Summary'].tolist()
rouge = evaluate.load('rouge')
for e in tqdm(df.iterrows()):
    content = e[1]['Content']
    content = content[:1024]
    summarization = summarization_pipeline_ft(content , min_length=5)[0]['summary_text']
    predictions.append(summarization)
    
results = rouge.compute(predictions=predictions,references=ground_truth)
print(results)

0it [00:00, ?it/s]

Your max_length is set to 128, but you input_length is only 110. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)
Your max_length is set to 128, but you input_length is only 58. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Your max_length is set to 128, but you input_length is only 79. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 128, but you input_length is only 126. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=63)
Your max_length is set to 128, but you input_length is only 72. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 128, but you input_length is only 73. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)


{'rouge1': 0.23428974441011538, 'rouge2': 0.06759631182667644, 'rougeL': 0.14956918054867135, 'rougeLsum': 0.18574934506428276}


In [14]:
predictions = []
ground_truth = df['Summary'].tolist()
rouge = evaluate.load('rouge')
for e in tqdm(df.iterrows()):
    content = e[1]['Content']
    content = content[:1024]
    summarization = summarization_pipeline_base(content , min_length=5)[0]['summary_text']
    predictions.append(summarization)
    
results = rouge.compute(predictions=predictions,references=ground_truth)
print(results)

0it [00:00, ?it/s]

Your max_length is set to 128, but you input_length is only 110. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)
Your max_length is set to 128, but you input_length is only 58. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Your max_length is set to 128, but you input_length is only 79. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 128, but you input_length is only 126. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=63)
Your max_length is set to 128, but you input_length is only 72. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 128, but you input_length is only 73. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)


{'rouge1': 0.2968307730250321, 'rouge2': 0.11381039251022732, 'rougeL': 0.1846079764801058, 'rougeLsum': 0.23170666052950833}


#### Hinge Loss

In [ ]:
from torch.nn import HingeEmbeddingLoss
loss_fn = HingeEmbeddingLoss()

In [ ]:
from torch.optim import AdamW
# fine-tuning BART
optimizer = AdamW(model.parameters() , lr=5e-5)
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
from tqdm.auto import tqdm

model.train()

for epoch in range(num_epochs):
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)